<h3 >This Notebook will be used for Data Science Capstone Project from <b>Coursera</b></h3>

In [1]:
import pandas as pd
import numpy as np


In [2]:
print("Hello Capstone project Course!")

Hello Capstone project Course!


In [3]:
import requests
from bs4 import BeautifulSoup

<h3>Import data from the url and parse the data using beautiful soup</h3>

In [4]:
html_toronto=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
html_parsed=BeautifulSoup(html_toronto,"html5lib")


<h3>Extract data from the html page and store in dataframe</h3>

In [5]:
df=pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood'])
for row in html_parsed.find('tbody').find_all('td'):
    postal=row.p.b.text
    temp=row.p.b.next_sibling.next_sibling.contents
    if row.p.b.next_sibling.next_sibling.text=='Not assigned':
        neighborhood='Not assigned'
        borough='Not assigned'
    else:
        borough=temp[0].string        
        neighborhood=row.p.b.next_sibling.next_sibling.text[row.p.b.next_sibling.next_sibling.text.find('(')+1:-1].replace('/',',')
    df=df.append({'Postal Code':postal,'Borough':borough,'Neighbourhood':neighborhood},ignore_index=True)
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo..."


In [6]:
df=df[df['Borough']!='Not assigned']
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Enclave of M4L
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [7]:
df['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [8]:
df['Neighbourhood'].value_counts()

St. James Town                                       1
First Canadian Place , Underground city              1
Thorncliffe Park                                     1
Moore Park , Summerhill East                         1
New Toronto , Mimico South , Humber Bay Shores       1
                                                    ..
Kensington Market , Chinatown , Grange Park          1
Kennedy Park , Ionview , East Birchmount Park        1
Wexford , Maryvale                                   1
Enclave of L4W                                       1
Cliffside , Cliffcrest , Scarborough Village West    1
Name: Neighbourhood, Length: 103, dtype: int64

<h3>Loading latitude and longitude data from csv file</h3>

In [9]:
df_lat_lang=pd.read_csv('Geospatial_Coordinates.csv')
df_lat_lang


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<h3> Merging Neighbourhood data and coordinates together</h3>

In [10]:
df_final=pd.merge(df,df_lat_lang,on='Postal Code')
df_final

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


In [11]:
df_final.isna().sum()

Postal Code      0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [12]:
import folium

In [13]:
latitude=43.6532
longitude=-79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = 'RSWCQJ2TTROEGUXB555H2FP0YFV1UVUXEVXC2JAY1RIHEAIS' # your Foursquare ID
CLIENT_SECRET = 'YV1LPP2GQHB1VBCXOCNO2SSHM34KVAYJUWEMLVRXLKDSJAF0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RSWCQJ2TTROEGUXB555H2FP0YFV1UVUXEVXC2JAY1RIHEAIS
CLIENT_SECRET:YV1LPP2GQHB1VBCXOCNO2SSHM34KVAYJUWEMLVRXLKDSJAF0


<h3>Downloading venues for each neighbouhood in toronto</h3>

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues=getNearbyVenues(df_final['Neighbourhood'],df_final['Latitude'],df_final['Longitude'])

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(2112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [18]:
print(toronto_venues['Venue Category'].nunique() , "unique categories")

272 unique categories


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt),4,4,4,4,4,4
"Alderwood , Long Branch",8,8,8,8,8,8
"Bathurst Manor , Wilson Heights , Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
Willowdale)Sout,35,35,35,35,35,35
Willowdale)Wes,5,5,5,5,5,5
Woburn,4,4,4,4,4,4


In [20]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_onehot.shape

(2112, 272)

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(100, 272)


,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale)Sout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0
96,Willowdale)Wes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.2,0.000000,0.0,0.0,0.0,0.0,0.0


<h3> Getting top 5 venue categories in all neighbourhood</h3>

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt)----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4         Mexican Restaurant  0.00


----Alderwood , Long Branch----
            venue  freq
0     Pizza Place  0.25
1     Coffee Shop  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor , Wilson Heights , Downsview North----
                       venue  freq
0                Coffee Shop  0.09
1                       Bank  0.09
2                  Pet Store  0.05
3  Middle Eastern Restaurant  0.05
4              Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park , Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.12
1           

                  venue  freq
0                  Café  0.15
1           Pizza Place  0.08
2   American Restaurant  0.08
3  Fast Food Restaurant  0.08
4            Restaurant  0.08


----North Park , Maple Leaf Park , Upwood Park----
                        venue  freq
0                      Bakery  0.25
1  Construction & Landscaping  0.25
2                        Park  0.25
3            Basketball Court  0.25
4                 Yoga Studio  0.00


----North Toronto West----
                 venue  freq
0          Coffee Shop  0.12
1       Clothing Store  0.12
2          Yoga Studio  0.06
3  Sporting Goods Shop  0.06
4                 Park  0.06


----Northwood Park , York University----
                   venue  freq
0     Miscellaneous Shop  0.17
1         Massage Studio  0.17
2   Caribbean Restaurant  0.17
3                    Bar  0.17
4  Vietnamese Restaurant  0.17


----Old Mill South , King's Mill Park , Sunnylea , Humber Bay , Mimico NE , The Queensway East , Royal York South Eas

         venue  freq
0  Pizza Place   0.2
1  Coffee Shop   0.2
2      Butcher   0.2
3  Supermarket   0.2
4     Pharmacy   0.2


----Woburn----
                   venue  freq
0            Coffee Shop  0.50
1  Korean BBQ Restaurant  0.25
2           Soccer Field  0.25
3            Yoga Studio  0.00
4    Monument / Landmark  0.00


----Woodbine Heights----
          venue  freq
0          Park   0.2
1    Beer Store   0.2
2   Video Store   0.2
3  Skating Rink   0.2
4   Curling Ice   0.2


----York Mills West----
                 venue  freq
0                 Park   0.5
1    Convenience Store   0.5
2          Yoga Studio   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0




<h3>Storing top 10 venue categories for all neighbourhood in dataframe</h3>

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt),Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood , Long Branch",Pizza Place,Pharmacy,Pub,Gym,Coffee Shop,Sandwich Place,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pet Store,Supermarket,Sushi Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Deli / Bodega,Mobile Phone Shop,Sandwich Place
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Thai Restaurant,Pharmacy,Pizza Place,Pub,Café,Restaurant,Butcher


In [26]:
from sklearn.cluster import KMeans

<h3>Appplying K mean clustering to the data</h3>

In [27]:
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       1, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2,
       3, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0,
       0, 2, 0, 2, 3, 0, 0, 0, 2, 0, 0, 3])

In [28]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [29]:
toronto_merged = df_final

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Bus Stop,Park,Food & Drink Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Pub,Bakery,Park,Theater,Breakfast Spot,Café,Gym / Fitness Center,Restaurant,Performing Arts Venue
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Furniture / Home Store,Boutique,Escape Room,Ethiopian Restaurant,Electronics Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Bar,Italian Restaurant,Beer Bar,Japanese Restaurant,Sandwich Place,Burrito Place
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,M3B,North York,Don Mills)Nort,43.745906,-79.352188,0.0,Gym,Construction & Landscaping,Japanese Restaurant,Caribbean Restaurant,Café,Concert Hall,Convenience Store,Event Space,Ethiopian Restaurant,Escape Room
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,2.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Flea Market,Pharmacy,Café,Bank,Gym / Fitness Center,American Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Hotel,Electronics Store,Theater


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [31]:
toronto_merged=toronto_merged[toronto_merged['Cluster Labels'].notna()]

<h3>Diplaying different clusters with different color</h3>

In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
your_choice=input("Enter your present postal code")


Enter your present postal codeM4B


In [55]:
temp=toronto_merged[toronto_merged['Postal Code']==your_choice].iloc[0,5:6].values[0]
toronto_merged[toronto_merged['Cluster Labels']==temp]['Neighbourhood']


1                                      Victoria Village
8                      Parkview Hill , Woodbine Gardens
17    Eringate , Bloordale Gardens , Old Burnhamthor...
19                                          The Beaches
50                                        Humber Summit
56    Del Ray , Mount Dennis , Keelsdale and Silvert...
70                                            Westmount
72                                       Willowdale)Wes
77    Kingsview Village , St. Phillips , Martin Grov...
79                                           Davisville
82            Clarks Corners , Tam O'Shanter , Sullivan
89    South Steeles , Silverstone , Humbergate , Jam...
90                       Steeles West , L'Amoreaux West
93                              Alderwood , Long Branch
Name: Neighbourhood, dtype: object